# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>
1. The appropriate test is Pearson's chi-square test to evaluate the hypothesis that a persons race and rate of callbacks are independent.
2. H0: Black sounding names and white sounding names have the same call back rate
   H1: Black sounding names have a lower callback rate than white sounding names

In [15]:
w = data[data.race=='w']
b = data[data.race=='b']
df = pd.DataFrame([data.race, data.call==1.0],['race','call']).transpose()
df['values'] = 1
table = pd.pivot_table(df, values='values',index='race',columns=['call'], aggfunc=np.sum)
table

call,False,True
race,,
b,2278,157
w,2200,235


We use a contingency table to see if we can use the CLT. Since none of the values are < 10 in the box we can use the approximation.

In [23]:
def chi2_test(x):
    chi2, p, dof, expected = stats.chi2_contingency(x)
    print(chi2,p,dof)
chi2_test(table)
odds = (235/2200)/(157/2278)
print(odds)

16.44902858418937 4.997578389963255e-05 1
1.5498841922408801


Since our p.value is < .05 we reject the null hypothesis. In addition our odds ratio is also 1.55. Therefore the odds of a person with a 'white' name recieving a callback are 1.55 times the odd that a person with a 'black' name will.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [31]:
def summarize(x):
    mu, s, n = x.mean(), x.std(), len(x)
    sem = stats.sem(x)
    CI = stats.norm.interval(.95, mu, sem)
    print(mu,s**2,sem,CI)
summarize(data[data.race=='w'].call)
summarize(data[data.race=='b'].call)

0.09650924056768417 0.08722897323814127 0.005985301397503016 (0.08477826539196101, 0.10824021574340734)
0.0644763857126236 0.06034365571350975 0.0049781434352911685 (0.05471940386957841, 0.07423336755566878)


White Names

sample mean = .1, sample var = .087, sample margin of error = .006, 95% CI: (.08664, .10635)

Black Names

sample mean = .06, sample var = .06, sample margin of error = .005, 95% CI: (.05628, .07266)

In order to minimize chance of racial discrimination in resume reviews, employers should probably remove the candidate names from the resumes. However, this analysis does not mean that race/name is the most important factor in callback success. There are a variety of other factors at play not being captured by this analysis. For example education and experience which are more than likely, the most important things on a resume when recieving a callback. To amend my analysis I could conduct another chi-square test to include the binomial factor of whether or not a candidate was college educated.